In [97]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv
from matplotlib import pyplot as plt
import cv2
import numpy as np
import json
import pandas as pd
import imutils

In [19]:
config_file = '/mmdetection/configs/detectors/missing_detector_latest_segm.py'
# download the checkpoint from model zoo and put it in `checkpoints/`
checkpoint_file = '/mmdetection/work_dirs/missing_detector_latest_segm_aug/epoch_16.pth'

In [20]:
# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cuda:0')

2021-02-24 06:56:34,109 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

missing keys in source state_dict: layer2.0.conv2.weight_diff, layer2.0.conv2.switch.weight, layer2.0.conv2.switch.bias, layer2.0.conv2.pre_context.weight, layer2.0.conv2.pre_context.bias, layer2.0.conv2.post_context.weight, layer2.0.conv2.post_context.bias, layer2.0.conv2.offset_s.weight, layer2.0.conv2.offset_s.bias, layer2.0.conv2.offset_l.weight, layer2.0.conv2.offset_l.bias, layer2.0.rfp_conv.weight, layer2.0.rfp_conv.bias, layer2.1.conv2.weight_diff, layer2.1.conv2.switch.weight, layer2.1.conv2.switch.bias, layer2.1.conv2.pre_context.weight, layer2.1.conv2.pre_context.bias, layer2.1.conv2.post_context.weight, layer2.1.conv2.post_context.bias, layer2.1.conv2.offset_s.weight, layer2.1.conv2.offset_s.bias, layer2.1.conv2.offset_l.weight, layer2.1.conv2.offset_l.bias, layer2.2.conv2.weight_diff, layer2.2.conv2.switch.weight, layer2

In [21]:
# test a single image
img = 'total2.jpg'
result = inference_detector(model, img)

In [22]:
# show the results
out=show_result_pyplot(model, img, result)

In [23]:
out[2][0]

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [90]:
def get_contour(im,out):
    #im=cv2.imread('total2.jpg')
    mask=np.zeros(im.shape[:2])
    pred_mask=out[2]
    for i in range(len(pred_mask)):
        mask=cv2.bitwise_or(mask.astype('uint8'),pred_mask[i].astype('uint8')*255)
    cnt, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)   
    return cnt

def convert_cp_cnt(pnts):
    cnts=[]
    for i in range(int(0.5*len(pnts))):
        cnts.append([[pnts[2*i],pnts[(2*i)+1]]])
    return np.array(cnts,np.int32)

def get_center(c):
    M = cv2.moments(c)
    if M["m00"]==0:
        return np.nan
    else:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        return cX,cY

def get_area(c):
    return cv2.contourArea(c)

def get_bbox(c):
    return cv2.boundingRect(c)

def get_rot_bbox(c):
    box = cv2.minAreaRect(c)
    box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
    return box

In [91]:
mode='train'
img_dir='/mmdetection/data/total-missing/images/'
ann_dir='/mmdetection/data/total-missing/annotations/missing_'+mode+'.json'
car_parts_ann='/mmdetection/data/carpart-total/total-missing/tm_'+mode+'_cp_0.json'
with open(ann_dir) as f:
    data=json.load(f)
with open(car_parts_ann) as f:
    car_parts_data=json.load(f)
    
carparts_names=['sli_side_turn_light', 'alloy_wheel', 'tail_gate', 'tli_tail_light', 'rwi_rear_windshield', 'fuel_tank_door', 'qpa_quarter_panel', 'lli_low_bumper_tail_light', 'mirror', 'fender', 'hood', 'flp_front_license_plate', 'fbe_fog_light_bezel', 'tyre', 'handle', 'fwi_windshield', 'rbu_rear_bumper', 'window', 'blp_back_license_plate', 'door', 'fbu_front_bumper', 'fli_fog_light', 'roof', 'pillar', 'hli_head_light', 'rocker_panel', 'grille']

column_name=[]

for i in range(4):
    for j in ['area','center','bbox','rot_bbox']:
        for k in carparts_names:
            column_name.append(str(k)+'_'+str(j)+'_'+str(i))

for i in range(5):
    for j in ['area','center','bbox','rot_bbox']:
        column_name.append('damage_'+str(j)+'_'+str(i))

column_name.extend(['width','height','total_carparts'])
df = pd.DataFrame(index=list(range(len(data['images']))),columns=column_name)

In [92]:
column_name

['sli_side_turn_light_area_0',
 'alloy_wheel_area_0',
 'tail_gate_area_0',
 'tli_tail_light_area_0',
 'rwi_rear_windshield_area_0',
 'fuel_tank_door_area_0',
 'qpa_quarter_panel_area_0',
 'lli_low_bumper_tail_light_area_0',
 'mirror_area_0',
 'fender_area_0',
 'hood_area_0',
 'flp_front_license_plate_area_0',
 'fbe_fog_light_bezel_area_0',
 'tyre_area_0',
 'handle_area_0',
 'fwi_windshield_area_0',
 'rbu_rear_bumper_area_0',
 'window_area_0',
 'blp_back_license_plate_area_0',
 'door_area_0',
 'fbu_front_bumper_area_0',
 'fli_fog_light_area_0',
 'roof_area_0',
 'pillar_area_0',
 'hli_head_light_area_0',
 'rocker_panel_area_0',
 'grille_area_0',
 'sli_side_turn_light_center_0',
 'alloy_wheel_center_0',
 'tail_gate_center_0',
 'tli_tail_light_center_0',
 'rwi_rear_windshield_center_0',
 'fuel_tank_door_center_0',
 'qpa_quarter_panel_center_0',
 'lli_low_bumper_tail_light_center_0',
 'mirror_center_0',
 'fender_center_0',
 'hood_center_0',
 'flp_front_license_plate_center_0',
 'fbe_fog_lig

In [93]:
car_parts_data['predict'][0].keys()

dict_keys(['image_name', 'damages'])

In [94]:
car_parts_data['predict'][0]['damages']['fender'][0]

[763,
 707,
 762,
 708,
 760,
 708,
 759,
 709,
 758,
 709,
 755,
 712,
 754,
 712,
 753,
 713,
 752,
 713,
 751,
 714,
 748,
 714,
 747,
 715,
 742,
 715,
 741,
 716,
 738,
 716,
 737,
 717,
 736,
 717,
 734,
 719,
 733,
 719,
 730,
 722,
 729,
 722,
 728,
 723,
 725,
 723,
 724,
 724,
 720,
 724,
 719,
 725,
 716,
 725,
 709,
 732,
 705,
 732,
 704,
 733,
 696,
 733,
 695,
 734,
 689,
 734,
 688,
 735,
 687,
 735,
 683,
 739,
 682,
 739,
 681,
 740,
 679,
 740,
 678,
 741,
 675,
 741,
 674,
 742,
 667,
 742,
 666,
 743,
 662,
 743,
 661,
 744,
 660,
 744,
 658,
 746,
 657,
 746,
 655,
 748,
 654,
 748,
 653,
 749,
 652,
 749,
 651,
 750,
 648,
 750,
 647,
 751,
 635,
 751,
 634,
 752,
 629,
 752,
 628,
 753,
 627,
 753,
 626,
 754,
 625,
 754,
 623,
 756,
 622,
 756,
 619,
 759,
 618,
 759,
 617,
 760,
 611,
 760,
 610,
 761,
 607,
 761,
 606,
 762,
 605,
 762,
 604,
 763,
 603,
 763,
 597,
 769,
 596,
 769,
 592,
 773,
 592,
 774,
 593,
 775,
 593,
 778,
 594,
 779,
 594,
 782,
 595

In [98]:
for i in range(len(data['images'])):
    col_out={}
    for cn in column_name:
        col_out[cn]=np.nan
    width=data['images'][i]['width']
    height=data['images'][i]['height']
    col_out['height']=height
    col_out['width']=width
    fn=data['images'][i]['file_name']
    for j in car_parts_data['predict']:
        #print(j['image_name'])
        if j['image_name']==fn:
            img=cv2.imread(img_dir+fn)
            result = inference_detector(model, img_dir+fn)
            out=show_result_pyplot(model, img_dir+fn, result)
            if len(out[2])>0:
                damage_cnt=get_contour(img,out)
                d_n=0
                for cd in damage_cnt:
                    col_out['damage_area_'+str(d_n)]=get_area(cd)
                    col_out['damage_center_'+str(d_n)]=get_center(cd)
                    col_out['damage_bbox_'+str(d_n)]=get_bbox(cd)
                    col_out['damage_rot_bbox_'+str(d_n)]=get_rot_bbox(cd)
                
            cp_ann=j['damages']
            for k in cp_ann.keys():
                cp_ann_ind=cp_ann[k]
                #print('#'*30)
                #print(k)
                #print(len(cp_ann_ind))
                for m in range(len(cp_ann_ind)):
                    cp_cnt=convert_cp_cnt(cp_ann_ind[m])
                    col_out[k+'_area_'+str(m)]=get_area(cp_cnt)
                    col_out[k+'_center_'+str(m)]=get_center(cp_cnt)
                    col_out[k+'_bbox_'+str(m)]=get_bbox(cp_cnt)
                    col_out[k+'_rot_bbox_'+str(m)]=get_rot_bbox(cp_cnt)
                    print(get_rot_bbox(cp_cnt))
                    #print(cp_cnt)
            col_out['total_carparts']=len(cp_ann.keys())
            #print(fn)
            #print(damage_cnt)
    #print(col_out)
    df.loc[i]=col_out

[[402.9094  244.20718]
 [888.71344 239.19888]
 [894.3601  786.9242 ]
 [408.5561  791.9325 ]]
[[500.37207 188.12819]
 [759.9495  175.46584]
 [761.0973  198.99522]
 [501.51984 211.65758]]
[[1043.4214   421.49057]
 [1053.24     339.6688 ]
 [1091.1738   344.22086]
 [1081.3552   426.04263]]
[[1128.  220.]
 [1141.  220.]
 [1141.  249.]
 [1128.  249.]]
[[ 819.9185    634.044   ]
 [1016.72644   -26.797302]
 [1197.8254     27.136536]
 [1001.01746   687.97784 ]]
[[492.4035  206.29994]
 [761.69086 168.33873]
 [768.20685 214.56114]
 [498.9195  252.52235]]
[[1068.   80.]
 [1094.   80.]
 [1094.   96.]
 [1068.   96.]]
[[1009.   52.]
 [1278.   52.]
 [1278.  885.]
 [1009.  885.]]
[[587.5487  761.0635 ]
 [769.46204 705.0088 ]
 [837.1498  924.6748 ]
 [655.23645 980.7295 ]]
[[1022.   78.]
 [1277.   78.]
 [1277.  873.]
 [1022.  873.]]
[[473. 358.]
 [793. 358.]
 [793. 717.]
 [473. 717.]]
[[1382.8518 2540.193 ]
 [4454.1626 1888.6287]
 [4647.5635 2800.2727]
 [1576.2529 3451.8372]]
[[1559.6185   496.85632]
 [3

[[405.83765 656.722  ]
 [749.5908  521.02997]
 [817.1133  692.08685]
 [473.3601  827.7789 ]]
[[1179.  589.]
 [1275.  589.]
 [1275.  919.]
 [1179.  919.]]
[[-12.89098   253.28778  ]
 [139.87576   230.04065  ]
 [153.62033   320.36212  ]
 [  0.8535919 343.60925  ]]
[[ 31.647072 570.41174 ]
 [ 85.470604 355.11765 ]
 [137.23532  368.05884 ]
 [ 83.41178  583.3529  ]]
[[ 60.069656 619.0044  ]
 [108.22943  500.02148 ]
 [481.27863  651.0176  ]
 [433.11884  770.0005  ]]
[[360. 719.]
 [369. 719.]
 [369. 720.]
 [360. 720.]]
[[ 49.86487  751.1892  ]
 [104.999985 642.      ]
 [438.64243  810.4729  ]
 [383.50732  919.6621  ]]
[[112.22044 692.7352 ]
 [138.57329 645.6101 ]
 [335.51324 755.74097]
 [309.1604  802.8661 ]]
[[ -23.244232  844.07623 ]
 [  63.323883  513.26227 ]
 [ 790.2556    703.48737 ]
 [ 703.6875   1034.3013  ]]
[[1083.  721.]
 [1274.  721.]
 [1274.  956.]
 [1083.  956.]]
[[-17.72443   342.91644  ]
 [170.7647    307.29645  ]
 [191.59247   417.51007  ]
 [  3.1033478 453.13007  ]]
[[703.711

KeyboardInterrupt: 

In [99]:
df[['sli_side_turn_light_rot_bbox_0']]

,sli_side_turn_light_rot_bbox_0
0,NaN
1,NaN
2,"[[2766.0, 1187.0], [2798.0, 1187.0], [2798.0, ..."
3,NaN
4,"[[1209.0, 494.0], [1225.0, 494.0], [1225.0, 51..."
...,...
1280,NaN
1281,NaN
1282,NaN
1283,NaN


In [104]:
car_parts_data['predict'][0]['damages'].keys()

dict_keys(['mirror', 'roof', 'pillar', 'fwi_windshield', 'fender', 'window', 'fli_fog_light'])

In [ ]:
len(data['images'])

In [41]:
len(out[2])

0